# 2D -> 3D Data Processing
Execution of this notebook is meant to follow `1_data_processing.ipynb`. 

The 

## Sample the data

In [ ]:
import os

# Constants
from PoseLifter import PoseLifter
from data_utils import crop_frame

# Choose dataset
dataset_path = "/home/florsanders/adl_ai_tennis_coach/data/tenniset"
write_path = "/home/georgetamer/3d_poses"
segments_path = os.path.join(dataset_path, "segments")
labels_path = os.path.join(dataset_path,"labels")

In [ ]:
from heuristics import keep_largest_volume_3D_pose_heuristic

pose_lifter = PoseLifter(
    crop_fn=crop_frame,
    dedup_heuristic_fn=keep_largest_volume_3D_pose_heuristic,
    dataset_path=dataset_path,
    write_path=write_path,
    duplicate_work=False)

In [ ]:
pose_lifter.extract_3d_poses()

In [ ]:
# %pip install numpy
# import numpy as np
# # Load the .npy file

# write_path = "/home/georgetamer/3d_poses"
# # sample_file = os.path.join(write_path, 'V009_0061_player_btm_pose_3d.npy')
# sample_file = os.path.join(labels_path, 'V010_0071_player_top_bbox.npy')
# data = np.load(sample_file, allow_pickle=True)
# # Print the contents of the file
# print(data.shape)

In [ ]:
# # visualize
# from mmpose.apis import visualize

# # extract first frame from segment
# img_path = os.path.join(write_path, '000000.jpg')
# keypoints = np.load(sample_file, allow_pickle=True)
# keypoint_scores = None

# metainfo = 'config/_base_/datasets/coco.py'

# visualize(
#     img_path,
#     keypoints,
#     keypoint_scores,
#     metainfo=metainfo,
#     show=True)